# 手書き数字認識

# 文字認識といえば、「MNISTデータセット」

その他のデータセット

３クラスに分類　→ あやめ

回帰　→ ボストンの住宅価格

2クラスに分類　→  タイタニックの生存者予想

# データセットの読み込み（テストデータ、訓練データ）と確認

In [1]:
from mnist import load_mnist
import pickle

In [2]:
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [3]:
x_train.shape

(60000, 784)

In [4]:
t_train.shape

(60000,)

In [5]:
x_test.shape

(10000, 784)

In [6]:
t_test.shape

(10000,)

# MNIST画像を表示させてみる

In [7]:
import numpy as np
from PIL import Image

In [8]:
def img_show(img):
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()
    
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)
img = x_train[0]
label = t_train[0]
print(label)

5


In [9]:
print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

img_show(img)

(784,)
(28, 28)


# ニューラルネットワークの推論処理

In [10]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y  =  exp_a / sum_exp_a
    
    return y

def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    return x_test, t_test


def init_network():
    with open("sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3
    y = softmax(a3)

    return y


In [11]:
# バッチ処理なしのものの速度を計測するためにtimeを導入
import time#time関数を取得
start = time.time()#開始時刻

x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    y = predict(network, x[i])
    p = np.argmax(y)
    if p == t[i]:
        accuracy_cnt += 1

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

end = time.time()#終了時刻
print(end-start, '秒')#終了時刻-開始時刻でかかった時間

Accuracy:0.9352
0.539849042892456 秒


# バッチ処理

In [12]:
x, _ = get_data()
network = init_network()
W1, W2, W3 = network['W1'], network['W2'], network['W3']

In [13]:
x.shape

(10000, 784)

In [14]:
x[0].shape

(784,)

In [15]:
W1.shape

(784, 50)

In [16]:
W2.shape

(50, 100)

In [17]:
W3.shape

(100, 10)

In [18]:
y.shape

(10,)

X  W1  W2  W3   →   Y  <br>
shape 100 * 784  784 * 50  50 * 100 100* 10  100 * 10

## バッチ処理を加えた処理

入力データのまとまりをバッチと言い、バッチ単位で推論処理を行うことで、計算を高速に行うことができる。

In [19]:
import time#time関数を取得
start = time.time()#開始時刻

x, t = get_data()
network = init_network()

batch_size = 100 # 付け加えた。
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size]
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))

end = time.time()#終了時刻
print(end-start, '秒')#終了時刻-開始時刻でかかった時間
# 結果は同じだが、計算速度が速くなる。
# バッチ処理なしの結果
# Accuracy:0.9352
# 時間:0.5786952972412109 秒

Accuracy:0.9352
0.13967490196228027 秒
